# Created by Diana Janik and Jan Markiewicz

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

In [2]:
x = np.load('classification_problem.npy')
y = []
class1 = np.array([0])
class1 = np.tile(class1, (1000, 1))
y.extend(class1)
class2 = np.array([1])
class2 = np.tile(class2, (1000, 1))
y.extend(class2)
class3 = np.array([2])
class3 = np.tile(class3, (1000, 1))
y.extend(class3)
y = np.asarray(y)
y = y.astype(np.long)
y = np.squeeze(y)
print(y)

[0 0 0 ... 2 2 2]


In [3]:
t_xs = torch.from_numpy(x)
t_ys = torch.from_numpy(y)
dataset = torch.utils.data.TensorDataset(t_xs, t_ys)
(train_set, valid_set, test_set) = torch.utils.data.random_split(dataset,(2000, 500, 500))
print(train_set[:][1])
# print train_set to prove that we have shuffled target values

tensor([2, 1, 1,  ..., 1, 0, 2])


In [4]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=int(len(train_set)/10))

In [5]:
x = Variable(torch.from_numpy(x))
y = Variable(torch.from_numpy(y))

In [6]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, 144)
        self.fc2 = nn.Linear(144, 81)
        self.fc3 = nn.Linear(81, 27)
        self.fc4 = nn.Linear(27, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [8]:
from IPython import display
import time
err_train=[]
err_valid=[]
outputlines = []
for epoch in range(2000):    
    for datum in train_loader:
        optimizer.zero_grad()
        (features,target) = datum
        pred = net(features)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        vpred  = net(valid_set[:][0])
        vloss  = criterion(vpred,valid_set[:][1])
        err_valid.append(vloss)
        pred  = net(train_set[:][0])
        loss  = criterion(pred,train_set[:][1])
        err_train.append(loss)
    if epoch%100 == 0:
        print("epoch %d %f %f " % (epoch, loss, vloss)) 
# I noticed that sometimes net could stack arount loss 1.1 and will not progress
# however I have no idea why sometimes it works and sometimes not

epoch 0 6.420899 6.711712 
epoch 100 0.996243 1.001011 
epoch 200 0.902342 0.900237 
epoch 300 0.908427 0.896722 
epoch 400 0.860533 0.850914 
epoch 500 0.851748 0.836192 
epoch 600 0.810493 0.797276 
epoch 700 0.853694 0.837289 
epoch 800 0.781884 0.772723 
epoch 900 0.785413 0.766365 
epoch 1000 0.780037 0.764227 
epoch 1100 0.859284 0.835818 
epoch 1200 0.772301 0.754897 
epoch 1300 0.760539 0.750496 
epoch 1400 0.869915 0.852200 
epoch 1500 0.743820 0.744599 
epoch 1600 0.760210 0.753337 
epoch 1700 0.805545 0.788871 
epoch 1800 0.808323 0.791571 
epoch 1900 0.770376 0.758092 


In [9]:
test_x = test_set[:][0]
test_y = test_set[:][1]

In [10]:
pred = []
correct_predictions = 0
for i in range(0, len(test_x)):
    tmp = net(test_x[i])
    pred.append(tmp.argmax(0))
    if pred[i] == test_y[i]:
        correct_predictions += 1
print("test accuracy: " + str(correct_predictions/len(test_x) * 100)+"%")

test accuracy: 68.4%
